<h1> TAN map based Audio Steganography Implementation </h1>

This notebook implements the Logistic Tan Map Based Audio Steganography method proposed in the paper "Logistic Tan Map Based Audio Steganography" by Marwa Tarek Elkandoz and Wassim Alexan in November 2019.
Link: https://ieeexplore.ieee.org/document/8959683

This implementation is created as a part of Henrik Hansen Stormyhr's Master's Thesis at the Norwegian University of Science and Technology.

AI Declaration:
Most of the code in this Implementation is generated by either Microsoft Copilot or ChatGPT. Some modifications have naturally been made, and the code has been pieced together to implement the 2D TAN map based steganography method.

The AES-128 implementation was largely generated by Microsoft CoPilot.
The Tan Logistic Map implementation was largely generated by both ChatGPT and Microsoft CoPilot. The AES-128 implementation was largely generated by CoPilot, while the rest of the code has been generated and modified by both ChatGPT and CoPilot.

Naturally I also made my own modifications to some of the generated code to make it work properly.

Python version 3.13.2 was used for this project.

In [1]:
# AES-128 Depencency
%pip install pycryptodome

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# TAN map dependencies
%pip install numpy
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install tabulate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


AES-128 Implementation:

In [1]:
# Importing AES Dependencies
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

In [2]:
# Message to be embedded in the cover file
secret_message = "Hello World!"

In [3]:
# Code for encrypting the secret message with AES-128

# Generate 128 bit (16 byte) key
key = get_random_bytes(16)

# Generate initialization vector
iv = get_random_bytes(16)

# Make cipher object for encryption
aes128enc = AES.new(key, AES.MODE_CFB, iv=iv)

# Encrypt message
plaintext = secret_message
ciphertext = aes128enc.encrypt(plaintext.encode('utf-8'))

print(f"Key: {key.hex()}")
print(f"IV: {iv.hex()}")
print(f"Ciphertext: {ciphertext.hex()}")

secret_message = ciphertext

Key: f6144ab6cf6d5ad6d2f3a34e42e17a31
IV: 064b581d6464165ed2bd6aa5155dd06a
Ciphertext: ed8e7633da833ec247405673


In [4]:
# Decryption test

# Make new cihper object for decryption
aes128dec = AES.new(key, AES.MODE_CFB, iv=iv)

# Decrypt
decrypted_message = aes128dec.decrypt(secret_message)

# Encode
decrypted_message_string = decrypted_message.decode('utf-8')

print(decrypted_message_string)

Hello World!


TAN map implementation:

To ensure that we had the same implementation as the paper we decided that we needed to reproduce the results from their Table 1. This table shows the values they produced after multiplying the output from their implementation of the 2 dimensional TAN map. We saw in the table that the papers starting values had to have been 0.3 for both x and y, so we started by copying these. Then we had to guess the r value. We guessed values from 0.1-0.9 in increments of 0.1 and intitially found that 0.9 produced the same two first outputs as table 1 from the paper. We eventually discovered that the rest of the values likely didnt match because of a spelling mistake in the y function of the tan map. Xn should have been Xn + 1 in the y function to offsett the iterations of the two functions. More information about how this was discovered can be read in my thesis. After correcting this the output values for the next iterations also looked more familiar to the tan paper. However, we had negative outputs that made our values differ from the paper. We then noticed that the papers implementation likely just got rid of any negative values, and after adding this to our implementation we were able to produce the same values as the paper proposing the method. This made us confident that we had likely replicated their exact implementation correctly. Only the code for the final correct and working implementation has been included here.

This cell contains just the 2D TAN map code to show the correctly replicated values from table 1:

In [5]:
import numpy as np

def tan_logistic_map(x, y, r, iterations=100):
    results = []
    print_list = []
    for _ in range(iterations):
        arg_x = (np.pi * r * y + 3) * (x * (1 - x))
        x = np.tan(arg_x)  # Update x first

        arg_y = (np.pi * r * x + 3) * (y * (1 - y))
        y = np.tan(arg_y)  # Use updated x for y
        

        # Scale by 10^7 and extract whole number parts
        if x >= 0:
            scaled_x = x * 1e7
            whole_x = int(scaled_x)
            #print(whole_x)
            print_list.append(whole_x)

        if y >= 0:
            scaled_y = y * 1e7
            whole_y = int(scaled_y)
            #print(whole_y)
            print_list.append(whole_y)

        # scaled_x, scaled_y = x * 1e7, y * 1e7
        # whole_x, whole_y = int(scaled_x), int(scaled_y)

        # results.append((scaled_x, scaled_y, whole_x, whole_y))
    

    for i in range(0, len(print_list), 2):
        print(" - ".join([str(num) for num in print_list[i:i+2]]))


# Run the map with r = 0.9 for both equations
tan_logistic_map(x=0.3, y=0.3, r=0.9, iterations=100)


10465258 - 30235701
7346647 - 6264212
13410269 - 26680129
5695430 - 102796485
52553839 - 1431521
14999801 - 12300396
68126210 - 10239116
11570469 - 2097758
1397419 - 12608451
1796157 - 33663864
19618630 - 6133295
5349739 - 18278649
13332560 - 6283074
9533356 - 40742363
7538058 - 36926662
467180 - 2801694
1705097 - 8461451
9555716 - 9172838
2420387 - 2870639
8410427 - 19679253
22045318 - 31366744
7719688 - 2074977
981178 - 20842285
10031083 - 2551167
6300187 - 7637032
3411520 - 10321794
40770111 - 27032924
5034370 - 62826151
15127024 - 4072585
7934524 - 2787841
5368238 - 12820554
11108435 - 1694556
2452444 - 8164515
14993523 - 18933550
290078 - 18276437
2341965 - 9197867
15728125 - 6123274
5954560 - 81812831
117123 - 11824858
735582 - 447173
345323463 - 22797553
25013752 - 10873444
4743397 - 4703960
4460115 - 17908551
311959972 - 1831911
1182720 - 10271768
43573369 - 10083378
3621071 - 7936452
4291624 - 49255059
914828 - 154935414
9291847 - 392180
54225289 - 8265872
10594652 - 11601691


Below is Table 1 from the TAN map paper for comparison of the output values:

![The TAN map paper's Table 1](TAN_map_fig1.png)

We noticed that the tan paper for some reason does not contain value 1431521, but rather skips directly to the next value 14999801. We strongly suspect that this is another typo in the tan paper since all other values match perfectly. Perhaps caused by both values starting with 14.

In the cells below we are applying the tan map to embed a secret unencrypted message:

In [6]:
import numpy as np
import wave
import scipy.io.wavfile as wav
import os

def tan_log_map_whole_numbers(x, y, r1, r2, required_count):
    results = []
    
    for _ in range(required_count):
        arg_x = (np.pi * r1 * y + 3) * (x * (1 - x))
        x = np.tan(arg_x)
        
        arg_y = (np.pi * r2 * x + 3) * (y * (1 - y))
        y = np.tan(arg_y)
        
        whole_x, whole_y = int(x * 1e7), int(y * 1e7)

        if whole_x >= 0:
            results.append(whole_x)
        
        if whole_y >= 0:
            results.append(whole_y)

    print(results)
    
    return results

def embed_message(audio_path, output_path, message, chaotic_indices):
    with wave.open(audio_path, 'rb') as audio:
        params = audio.getparams()
        frames = audio.readframes(audio.getnframes())

    audio_data = np.frombuffer(frames, dtype=np.int16).copy()
    original_audio_data = audio_data.copy()  # Keep original for debugging

    # Convert message to bits, appending a null terminator (`\0`)
    message += '\0'  # Append null terminator
    message_bits = ''.join(f'{ord(c):08b}' for c in message)
    print("Original message bits:", message_bits)

    # Ensure we have enough chaotic indices
    indices = chaotic_indices[:len(message_bits)]

    # Embed message bits into LSB of selected samples
    for i, bit in enumerate(message_bits):
        sample_index = indices[i]
        audio_data[sample_index] = (audio_data[sample_index] & ~1) | int(bit)

    # Debug: Print first 20 modified samples
    for i in range(min(20, len(indices))):
        sample_index = indices[i]
        print(f"Sample {sample_index}: {original_audio_data[sample_index]} -> {audio_data[sample_index]}")

    with wave.open(output_path, 'wb') as output:
        output.setparams(params)
        output.writeframes(audio_data.tobytes())


def extract_message(audio_path, chaotic_indices):
    with wave.open(audio_path, 'rb') as audio:
        frames = audio.readframes(audio.getnframes())

    audio_data = np.frombuffer(frames, dtype=np.int16).copy()

    extracted_bits = []
    extracted_chars = []

    for idx in chaotic_indices:
        extracted_bits.append(str(audio_data[idx] & 1))
        
        # Convert bits into characters every 8 bits
        if len(extracted_bits) % 8 == 0:
            char = chr(int(''.join(extracted_bits[-8:]), 2))
            if char == '\0':  # Stop at null terminator
                break
            extracted_chars.append(char)

    return ''.join(extracted_chars)

def generate_chaotic_indices(estimated_bits, audio_samples, x=0.3, y=0.3, r1=0.9, r2=0.9):
    required_count = estimated_bits + 64  # Generate extra indices for safety
    chaotic_numbers = tan_log_map_whole_numbers(x, y, r1, r2, required_count)  
    chaotic_indices = np.mod(chaotic_numbers, audio_samples)
    return chaotic_indices

# Example usage:
audio_input = "COVER_Female1_Sentence1.wav"
audio_output = "stegofiles/output.wav"
message = "Hello"

# Open the audio file to get the number of samples
with wave.open(audio_input, 'rb') as audio:
    num_samples = audio.getnframes()

# Generate chaotic indices based on your map
chaotic_indices = generate_chaotic_indices(len(message) * 8, num_samples)

# Embed the message
embed_message(audio_input, audio_output, message, chaotic_indices)

# Extract the message
extracted_message = extract_message(audio_output, chaotic_indices)
print("Extracted Message:", extracted_message)

[10465258, 30235701, 7346647, 6264212, 13410269, 26680129, 5695430, 102796485, 52553839, 1431521, 14999801, 12300396, 68126210, 10239116, 11570469, 2097758, 1397419, 12608451, 1796157, 33663864, 19618630, 6133295, 5349739, 18278649, 13332560, 6283074, 9533356, 40742363, 7538058, 36926662, 467180, 2801694, 1705097, 8461451, 9555716, 9172838, 2420387, 2870639, 8410427, 19679253, 22045318, 31366744, 7719688, 2074977, 981178, 20842285, 10031083, 2551167, 6300187, 7637032, 3411520, 10321794, 40770111, 27032924, 5034370, 62826151, 15127024, 4072585, 7934524, 2787841, 5368238, 12820554, 11108435, 1694556, 2452444, 8164515, 14993523, 18933550, 290078, 18276437, 2341965, 9197867, 15728125, 6123274, 5954560, 81812831, 117123, 11824858, 735582, 447173, 345323463, 22797553, 25013752, 10873444, 4743397, 4703960, 4460115, 17908551, 311959972, 1831911, 1182720, 10271768, 43573369, 10083378, 3621071, 7936452, 4291624, 49255059, 914828, 154935414, 9291847, 392180, 54225289, 8265872, 10594652, 11601691,

Finally, we apply the entire method as described in the paper. First encrypting our secret message with AES-128, then using the 2D TAN map to embed it into an audio file before finally extracting and decrypting it again:

This code sometimes gets errors when trying to convert the decoded AES-128 encrypted message to UTF-8 (line below comment "# Encode"). If this happens, just rerun it. The code still functions as a proof of work, and this is likely not too complicated to fix. However, it was not something that we needed to fix for our purposes.

In [16]:
# Example usage of entire TAN method as described in its paper

# Secret message to be embedded
secret_message = "Hello beautiful world"

# Code for encrypting the secret message with AES-128:

# Generate 128 bit (16 byte) key
key = get_random_bytes(16)

# Generate initialization vector
iv = get_random_bytes(16)

# Make cipher object for encryption
aes128enc = AES.new(key, AES.MODE_CFB, iv=iv)

# Encrypt message
plaintext = secret_message
ciphertext = aes128enc.encrypt(plaintext.encode('utf-8'))

print(f"Key: {key.hex()}")
print(f"IV: {iv.hex()}")
print(f"Ciphertext: {ciphertext.hex()}")

secret_message = ciphertext.hex()


# Code for applying logistic tan map and embedding the secret message in the cover audio file:

audio_input = "COVER_Female1_Sentence1.wav"
audio_output = "stegofiles/output.wav"
message = secret_message

# Open the audio file to get the number of samples
with wave.open(audio_input, 'rb') as audio:
    num_samples = audio.getnframes()

# Generate chaotic indices based on your map
chaotic_indices = generate_chaotic_indices(len(message) * 8, num_samples)

# Embed the message
embed_message(audio_input, audio_output, message, chaotic_indices)

# Extract the message (extract the AES-128 cihpertext that is still encrypted)
extracted_message = extract_message(audio_output, chaotic_indices)
print("Extracted Message (AES-128 ciphertext):", extracted_message)


# Code for decrypting the AES-128 ciphertext to read the extracted secret message:

# Make new cihper object for decryption
aes128dec = AES.new(key, AES.MODE_CFB, iv=iv)

# Decrypt
decrypted_message = aes128dec.decrypt(bytes.fromhex(extracted_message))

# Encode
decrypted_message_string = decrypted_message.decode('utf-8')

print("Decrypted message: " + str(decrypted_message_string))



Key: 127c4c6cfaa81ffcd653c6e7d91079fb
IV: d36b5f6afbb664b88c31c8c41d8a0a8b
Ciphertext: f4c8a519b246c32558a488f7f22426d9a36620fd08
[10465258, 30235701, 7346647, 6264212, 13410269, 26680129, 5695430, 102796485, 52553839, 1431521, 14999801, 12300396, 68126210, 10239116, 11570469, 2097758, 1397419, 12608451, 1796157, 33663864, 19618630, 6133295, 5349739, 18278649, 13332560, 6283074, 9533356, 40742363, 7538058, 36926662, 467180, 2801694, 1705097, 8461451, 9555716, 9172838, 2420387, 2870639, 8410427, 19679253, 22045318, 31366744, 7719688, 2074977, 981178, 20842285, 10031083, 2551167, 6300187, 7637032, 3411520, 10321794, 40770111, 27032924, 5034370, 62826151, 15127024, 4072585, 7934524, 2787841, 5368238, 12820554, 11108435, 1694556, 2452444, 8164515, 14993523, 18933550, 290078, 18276437, 2341965, 9197867, 15728125, 6123274, 5954560, 81812831, 117123, 11824858, 735582, 447173, 345323463, 22797553, 25013752, 10873444, 4743397, 4703960, 4460115, 17908551, 311959972, 1831911, 1182720, 10271768, 4